<a href="https://colab.research.google.com/github/Karan0322/Covid19-data-visualiztion/blob/main/Covid19data_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup as soup
from datetime import date, datetime
from urllib.request import Request, urlopen
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
import gc
import warnings
warnings.filterwarnings("ignore")
from pandas_profiling import ProfileReport

**Web Scrapping**

In [ ]:
today=datetime.now()
yesterday_str = "%s %d %d" %(date.today().strftime("%b"), today.day-1,today.year)
yesterday_str

'Jun 23 2022'

In [ ]:
url='https://www.worldometers.info/coronavirus/#countries'


In [ ]:
req=Request(url,headers={'User-Agent':'MOzilla/5.0'})
webpage= urlopen(req)
print(webpage)

In [ ]:
page_soup=soup(webpage,'html.parser')

In [ ]:
table= page_soup.findAll("table",{"id":"main_table_countries_yesterday"})
containers=table[0].findAll("tr",{"style":""})
title = containers[0]
del containers[0]

In [ ]:
all_data=[]
clean = True
for country in containers:
  country_data=[]
  country_container = country.findAll("td")
  if country_container[1].text== "China":
    continue
  for i in range (1,len(country_container)):
    final_feature = country_container[i].text
    if clean :
      if i != 1 and i != len(country_container)-1:
        final_feature = final_feature.replace(",","")
        if final_feature.find("+") != -1:
          final_feature = final_feature.replace("+","")
          final_feature=float(final_feature)
        elif final_feature.find("-") != -1:
          final_feature = final_feature.replace("-","")
          final_feature = float(final_feature)
    if final_feature=='N/A':
      final_feature = 0
    elif final_feature == "" or final_feature == " ":
      final_feature=-1
    country_data.append(final_feature)
  all_data.append(country_data) 


In [ ]:
all_data

[['World',
  '547360496',
  732726.0,
  '6347556',
  1675.0,
  '522665239',
  478742.0,
  '18347701',
  '36338',
  '70221',
  '814.3',
  -1,
  -1,
  -1,
  'All',
  '\n',
  -1,
  -1,
  -1,
  -1,
  -1],
 ['USA',
  '88566961',
  96702.0,
  '1040236 ',
  266.0,
  '84257913',
  84760.0,
  '3268812',
  '3074',
  '264514',
  '3107',
  '1045222702',
  '3121658',
  '334829357 ',
  'North America',
  '4',
  '322',
  '0',
  '289',
  '0.8',
  '9,763'],
 ['India',
  '43365016',
  17419.0,
  '524954 ',
  13.0,
  '42749056',
  13029.0,
  '91006',
  '698',
  '30826',
  '373',
  '859493387',
  '610976',
  '1406754645 ',
  'Asia',
  '32',
  '2680',
  '2',
  '12',
  '0.01',
  '65'],
 ['Brazil',
  '31963736',
  69231.0,
  '669958 ',
  346.0,
  '30554684',
  21816.0,
  '739094',
  '8318',
  '148297',
  '3108',
  '63776166',
  '295892',
  '215538383 ',
  'South America',
  '7',
  '322',
  '3',
  '321',
  '2',
  '3,429'],
 ['France',
  '30434451',
  79852.0,
  '149268 ',
  40.0,
  '29446664',
  25064.0,
  '8

In [ ]:
#create dataframe
df=pd.DataFrame(all_data)
df.drop([15,16,17,18,19,20], inplace=True, axis=1)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,World,547360496,732726.0,6347556,1675.0,522665239,478742.0,18347701,36338,70221,814.3,-1,-1,-1,All
1,USA,88566961,96702.0,1040236,266.0,84257913,84760.0,3268812,3074,264514,3107,1045222702,3121658,334829357,North America
2,India,43365016,17419.0,524954,13.0,42749056,13029.0,91006,698,30826,373,859493387,610976,1406754645,Asia
3,Brazil,31963736,69231.0,669958,346.0,30554684,21816.0,739094,8318,148297,3108,63776166,295892,215538383,South America
4,France,30434451,79852.0,149268,40.0,29446664,25064.0,838519,869,464235,2277,271490188,4141206,65558242,Europe


In [ ]:
column_labels=["Country","Total Cases","New Cases","Total Deaths","New Deaths","Total Recovered","New Recovered","Active Cases","Serious/Critical","Tot Cases/1M","Tot Deaths/1M","Total Tests","Tests/1M pop","Population","Continent"]
df.columns= column_labels

In [ ]:
df.head()

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Tot Cases/1M,Tot Deaths/1M,Total Tests,Tests/1M pop,Population,Continent
0,World,547360496,732726.0,6347556,1675.0,522665239,478742.0,18347701,36338,70221,814.3,-1,-1,-1,All
1,USA,88566961,96702.0,1040236,266.0,84257913,84760.0,3268812,3074,264514,3107,1045222702,3121658,334829357,North America
2,India,43365016,17419.0,524954,13.0,42749056,13029.0,91006,698,30826,373,859493387,610976,1406754645,Asia
3,Brazil,31963736,69231.0,669958,346.0,30554684,21816.0,739094,8318,148297,3108,63776166,295892,215538383,South America
4,France,30434451,79852.0,149268,40.0,29446664,25064.0,838519,869,464235,2277,271490188,4141206,65558242,Europe


In [ ]:
for label in df.columns:
  if label != "Country" and label != "Continent":
    df[label]=pd.to_numeric(df[label])

In [ ]:
df["%INC Cases"]=(df["New Cases"]/df["Total Cases"])*100
df["%INC Deaths"]=(df["New Deaths"]/df["Total Deaths"])*100
df["%INC Recovered"]=(df["New Recovered"]/df["Total Recovered"])*100

In [ ]:
df.head()

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Tot Cases/1M,Tot Deaths/1M,Total Tests,Tests/1M pop,Population,Continent,%INC Cases,%INC Deaths,%INC Recovered
0,World,547360496,732726.0,6347556,1675.0,522665239,478742.0,18347701,36338,70221.0,814.3,-1,-1,-1,All,0.133865,0.026388,0.091596
1,USA,88566961,96702.0,1040236,266.0,84257913,84760.0,3268812,3074,264514.0,3107.0,1045222702,3121658,334829357,North America,0.109185,0.025571,0.100596
2,India,43365016,17419.0,524954,13.0,42749056,13029.0,91006,698,30826.0,373.0,859493387,610976,1406754645,Asia,0.040168,0.002476,0.030478
3,Brazil,31963736,69231.0,669958,346.0,30554684,21816.0,739094,8318,148297.0,3108.0,63776166,295892,215538383,South America,0.216592,0.051645,0.071400
4,France,30434451,79852.0,149268,40.0,29446664,25064.0,838519,869,464235.0,2277.0,271490188,4141206,65558242,Europe,0.262374,0.026797,0.085117


**DATA ANALYSIS**

In [ ]:
cases=df[["Total Recovered","Active Cases","Total Deaths"]].loc[0]
cases_df=pd.DataFrame(cases).reset_index()
cases_df.columns=["Type","Total"]
cases_df["Percentage"]= np.round(100*cases_df["Total"]/np.sum(cases_df["Total"]),2)
cases_df["Virus"] = ["COVID-19" for i in range(len(cases_df))]
cases_df

,Type,Total,Percentage,Virus
0,Total Recovered,522665239,95.49,COVID-19
1,Active Cases,18347701,3.35,COVID-19
2,Total Deaths,6347556,1.16,COVID-19


In [ ]:
fig=px.bar(cases_df,x="Virus",y="Percentage",color="Type",hover_data=["Total"])
fig.show()

In [ ]:
continent_df=df.groupby("Continent").sum().drop("All")
continent_df=continent_df.reset_index()
continent_df

,Continent,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Tot Cases/1M,Tot Deaths/1M,Total Tests,Tests/1M pop,Population,%INC Cases,%INC Deaths,%INC Recovered
0,Africa,12278401,7077.0,255724,-16.0,10392767,4032.0,469750,999,1942159.0,18075.0,104917350,10653258,1403245876,2.186280,-13.383814,1.118595
1,Asia,159971511,139778.0,1429270,229.0,153951276,157772.0,3457396,10490,5881724.0,31562.0,2006413169,83471965,3235673375,10.995988,94.789726,4.099136
2,Australia/Oceania,9535860,37964.0,13654,49.0,9176707,35070.0,272957,136,2130191.0,6663.0,82446728,17900583,43513062,-23.246228,249.319169,78.125847
3,Europe,201441477,337174.0,1852281,480.0,191758770,146922.0,5136693,5846,15876542.0,114734.0,2698991740,202533800,748518998,6.715500,-6.396004,2.745593
4,North America,104583687,127425.0,1484856,390.0,98208479,99781.0,4878850,8422,7615131.0,53976.0,1169939439,94043280,598211603,2.320095,-166.991669,0.737507
5,South America,59169733,83156.0,1303343,383.0,55677876,34972.0,1323734,10368,2271636.0,31714.0,225095714,12952984,437767927,2.276808,100.174208,0.561990


In [ ]:
LOOK_AT=6
continent=continent_df.columns[1:14]

fig1=go.Figure()
c=0
for i in continent_df.index:
    if c < LOOK_AT:
      fig1.add_trace(go.Bar(name=continent_df['Continent'][i],x=continent,y=continent_df.loc[i][1:14]))
    else:
      break
    c=c+1


fig1.update_layout(title={"text":f'Continent wise analysis'},yaxis_type="log")
fig1.show()

In [ ]:
df=df.drop([len(df)-1])
country_df=df.drop([0])
country_df

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious/Critical,Tot Cases/1M,Tot Deaths/1M,Total Tests,Tests/1M pop,Population,Continent,%INC Cases,%INC Deaths,%INC Recovered
1,USA,88566961,96702.0,1040236,266.0,84257913,84760.0,3268812,3074,264514.0,3107.0,1045222702,3121658,334829357,North America,0.109185,0.025571,0.100596
2,India,43365016,17419.0,524954,13.0,42749056,13029.0,91006,698,30826.0,373.0,859493387,610976,1406754645,Asia,0.040168,0.002476,0.030478
3,Brazil,31963736,69231.0,669958,346.0,30554684,21816.0,739094,8318,148297.0,3108.0,63776166,295892,215538383,South America,0.216592,0.051645,0.071400
4,France,30434451,79852.0,149268,40.0,29446664,25064.0,838519,869,464235.0,2277.0,271490188,4141206,65558242,Europe,0.262374,0.026797,0.085117
5,Germany,27573585,119360.0,140560,98.0,26340200,33500.0,1092825,780,327042.0,1667.0,122332384,1450947,84312082,Europe,0.432878,0.069721,0.127182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,Wallis and Futuna,454,-1.0,7,-1.0,438,-1.0,9,-1,41839.0,645.0,20508,1889964,10851,Australia/Oceania,-0.220264,-14.285714,-0.228311
218,Macao,115,8.0,-1,-1.0,83,-1.0,32,-1,172.0,-1.0,5780,8668,666787,Asia,6.956522,100.000000,-1.204819
219,Micronesia,38,-1.0,-1,-1.0,33,-1.0,5,-1,324.0,-1.0,21923,186738,117400,Australia/Oceania,-2.631579,100.000000,-3.030303
220,Niue,10,-1.0,-1,-1.0,9,-1.0,1,-1,6068.0,-1.0,-1,-1,1648,Australia/Oceania,-10.000000,100.000000,-11.111111


In [ ]:
LOOK_AT=10
country=country_df.columns[1:14]

fig=go.Figure()
c=0
for i in country_df.index:
    if c < LOOK_AT:
      fig.add_trace(go.Bar(name=country_df['Country'][i],x=country,y=country_df.loc[i][1:14]))
    else:
      break
    c=c+1


fig.update_layout(title={"text":f'top {LOOK_AT} countries affected'},yaxis_type="log")
fig.show()